In [ ]:
#https://github.com/dreamquark-ai/tabnet
#pip install pytorch-tabnet

#https://github.com/automl/TabPFN
#pip install tabpfn

#pip install lightgbm

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
clinical = pd.read_excel("../data/20231124is0155-500NIHSSfinal.xlsx",sheet_name=0)
clinical.rename(columns = {'Unnamed: 0':"ID"}, inplace = True)

clinical = clinical.iloc[:,:-33]
#2D-EF 、 BUN 、iFS 、 HbA1c 
del_ls = [1,2,3,11,12,13,14,15,16,17]
print("delete",clinical.columns[del_ls])
clinical = clinical.drop(clinical.columns[del_ls],axis = 1)
clinical = clinical.rename(columns={"LVO-index>50%": "LVO-index", "Tx (1tPA, 2IA)": "Tx"})
clinical.iloc[[5,155,171],35]  = np.NaN
clinical.iloc[86,19] = 1
clinical.iloc[89,23] = 1
clinical.iloc[:,12:] = clinical.iloc[:,12:].apply(pd.to_numeric)


clinical.iloc[:,11:12] = clinical.iloc[:,11:12].fillna(clinical.iloc[:,11:12].quantile(0.5))
clinical.iloc[:,7:8] = clinical.iloc[:,7:8].fillna(clinical.iloc[:,7:8].quantile(0.5))
clinical.iloc[:,23:] = clinical.iloc[:,23:].fillna(clinical.iloc[:,23:].quantile(0.5))

clinical = clinical.drop(157,axis=0)
clinical

In [ ]:
#1441
radiomics = pd.read_csv("../data/1107_radiomics_output.csv")
#1424
del_ls = [1,2,3,4,5,6,7,8,9,10,11,15,16,17,18,21,22]
print("delete",radiomics.columns[del_ls])
radiomics = radiomics.drop(radiomics.columns[del_ls],axis = 1)
#1414+
radiomics = radiomics.dropna(axis=1)
radiomics
#1375
target = list(radiomics.var().index[radiomics.var()!=0])
target.insert(0,"ID")
radiomics = radiomics[target]
radiomics

In [ ]:
semantic = pd.read_csv("../data/0918_semantic_output.csv")
semantic

In [ ]:
fuse = pd.read_csv("../data/0416_fuse_radiomics_output.csv")
fuse = fuse.dropna(axis=1)
target = list(fuse.var().index[fuse.var()!=0])
target.insert(0,"ID")
fuse = fuse[target]
fuse

In [ ]:
from sklearn.utils import shuffle
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE


from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier

import shap

In [ ]:
def get_data(clinical,target_ls,target_num,info_ls=[],radiomics=None,semantic=None,fuse=None):
    y_name = clinical.columns[target_num]
    
    print("target is",list(clinical.columns[target_ls]))
    if radiomics is not None:
        merge = pd.merge(clinical[clinical.columns[target_ls+info_ls]], radiomics, left_on="ID", right_on="ID", how="inner")
    else:
        merge = clinical[clinical.columns[target_ls+info_ls]]
    if semantic is not None:
        merge = pd.merge(merge, semantic, left_on="ID", right_on="ID", how="inner")
    if fuse is not None:
        merge = pd.merge(merge, fuse, left_on="ID", right_on="ID", how="inner")
    print(merge.shape)
    merge = merge.dropna(axis=0)
    print(merge.shape)
    merge = merge.drop(clinical.columns[info_ls], axis=1)
    ylab = merge[y_name]
    data = merge.drop(["ID",y_name],axis = 1)
    data = data.astype('float64')
    print("shape of data is :",data.shape)
    return(data,ylab)
def featureIMP(data,model,modeltype="forest"):
    featureimp = pd.DataFrame()
    featureimp["feature"] = data.columns
    if modeltype in ["forest","lgbm","TabPFN","xgb"]:
        featureimp["imp"] = model.feature_importances_
    elif modeltype in ["catboost"]:
        featureimp["imp"] = model.get_feature_importance()
    featureimp = featureimp.sort_values(by='imp', ascending=False)
    featureimp = featureimp.reset_index(drop=True)
    return(featureimp)

In [ ]:
from sklearn import metrics
def where(df,l):
    return_list = []
    for i in range(len(df.columns)):
        if df.columns[i] in l:
            return_list.append(i)
    return(return_list)
def listgenerate(k,catnum,catremain):
    temp = []
    for i in range(k):
        for j in range(catnum):
            temp.append(i)
    for i in range(catremain):
        temp.append(i)
    temp = np.array(temp)
    random.shuffle(temp)
    return(temp)
        
        
def cv(data,y,k=5,cate=True):
    if(cate):
        datat = data[y==0]
        yt = y[y==0]
        catnum = sum(y==0)// k
        catremain = sum(y==0) % k
        listt = listgenerate(k,catnum,catremain)
       
        
        dataf = data[y==1]
        yf = y[y==1]        
        catnum = sum(y==1)// k
        catremain = sum(y==1) % k
        listf = listgenerate(k,catnum,catremain)
        
        outputdata = []
        outputy = []
        for i in range(k):
            kdata = pd.concat([datat[listt==i],dataf[listf==i]])
            outputdata.append(kdata)
            ky = pd.concat([yt[listt==i],yf[listf==i]])
            outputy.append(ky)

    else:
        catnum = len(y)// k
        catremain = len(y) % k
        lista = listgenerate(k,catnum,catremain)
        outputdata = []
        outputy = []
        for i in range(k):
            outputdata.append(data[lista==i])
            outputy.append(y[lista==i])
        
    return(outputdata,outputy)
def analyse(kdata,ky,model,smote=False,cat=False,sampling_strategy=0.56,modeltype="forest"):

    
    reshape_k_y = np.array([])
    reshape_k_predict = np.array([])
    reshape_k_prob = np.array([])
    shape = kdata[0].shape
    
    reshape_k_data = pd.DataFrame(columns = kdata[0].columns)
    reshape_k_shap = np.zeros((0, shape[1]))
    featureimp = pd.DataFrame({"feature":kdata[0].columns})

        
    for i in range(len(kdata)):
        traindata = pd.DataFrame([],columns = kdata[0].columns)
        trainy = pd.Series([],dtype=float)
        for j in range(len(kdata)):
            if i!=j:
                traindata = pd.concat([traindata,kdata[j]])
                trainy = pd.concat([trainy,ky[j]])
            else:
                testdata = kdata[j]
                testy = ky[j]
        
        if smote:
            cat=True
            sm = SMOTE(random_state=42,sampling_strategy=sampling_strategy,k_neighbors=3)
            traindata, trainy = sm.fit_resample(traindata, trainy)
            traindata = traindata.astype('float64')

        tempmodel = model
        if modeltype in ["forest","xgb","lgbm","TabPFN"]:
            tempmodel.fit(traindata, trainy)
            predict = tempmodel.predict(testdata)
            predict_porb = tempmodel.predict_proba(testdata)[:,1]
            
            #predict_train = tempmodel.predict(traindata)
        elif modeltype in ["catboost"]:
            tempmodel.fit(traindata, trainy, eval_set=(testdata, testy), verbose=0, plot=False)
            predict = tempmodel.predict(testdata)
            predict_porb = tempmodel.predict_proba(testdata)[:,1]
            
            #predict_train = tempmodel.predict(traindata)
        elif modeltype in ["TabNet"]:
            tempmodel.fit(traindata.values, trainy,eval_set=[(testdata.values, testy)])
            predict = tempmodel.predict(testdata.values)
            predict_porb = tempmodel.predict_proba(testdata)[:,1]
            #predict_train = tempmodel.predict(traindata.values)
        else:
            print("warning,modeltype error.")

        explainer = shap.Explainer(tempmodel)
        shap_values = explainer.shap_values(testdata)
        reshape_k_data = pd.concat((reshape_k_data,kdata[i]))
        
        if modeltype in ["forest","lgbm"]:
            reshape_k_shap = np.concatenate((reshape_k_shap,shap_values[1]))
        elif modeltype in ["xgb","catboost"]:
            reshape_k_shap = np.concatenate((reshape_k_shap,shap_values))
        
        temp_featureimp = featureIMP(testdata,tempmodel,modeltype=modeltype)
        reshape_k_y = np.concatenate((reshape_k_y,testy))
        reshape_k_predict = np.concatenate((reshape_k_predict,predict))
        reshape_k_prob = np.concatenate((reshape_k_prob,predict_porb))
        featureimp = featureimp.merge(temp_featureimp, how='inner', on='feature')

    return(reshape_k_y,reshape_k_predict,reshape_k_prob,featureimp,reshape_k_data,reshape_k_shap)


def CI_analyse(data,y,model,modeltype,k=3,cate=True,time=21,smote=True,record=False,plotname=""):

    all_data = pd.DataFrame(columns = data.columns)
    shape = data.shape
    all_shap = np.zeros((0, shape[1]))
    
    acc = []
    sen = []
    spe = []
    auc = []
    auprc = []
    l_fpr = []
    l_tpr = []
    l_precision = []
    l_recall = []
    CI_featureimp = pd.DataFrame({"feature":data.columns})
    for i in range(21):
        
        kdata,ky = cv(data,y,k=k,cate=cate)
        model_temp = model
        
        reshape_k_y,reshape_k_predict,reshape_k_prob,featureimp,reshape_k_data,reshape_k_shap = analyse(kdata,ky,model_temp,smote=smote,cat=True,sampling_strategy=1,modeltype=modeltype)
        
        all_data = pd.concat((all_data,reshape_k_data))
        all_shap = np.concatenate((all_shap,reshape_k_shap))
        
        
        CI_featureimp = CI_featureimp.merge(featureimp, how='inner', on='feature')
        ans_test=confusion_matrix(reshape_k_y, reshape_k_predict)
        fpr, tpr, thresholds = metrics.roc_curve(reshape_k_y, reshape_k_prob)
        precision , recall, thresholds = metrics.precision_recall_curve(reshape_k_y, reshape_k_prob , pos_label=1)
        
        acc.append(round((ans_test[0,0]+ans_test[1,1])/(ans_test[0,0]+ans_test[0,1]+ans_test[1,0]+ans_test[1,1]),3))
        sen.append(round(ans_test[1,1]/(ans_test[1,0]+ans_test[1,1]),3))
        spe.append(round(ans_test[0,0]/(ans_test[0,0]+ans_test[0,1]),3))
        auc.append(round(metrics.auc(fpr, tpr),3))
        auprc.append(round(metrics.auc(recall , precision),3))
        l_fpr.append(fpr)
        l_tpr.append(tpr)
        l_precision.append(precision)
        l_recall.append(recall)

    CI_featureimp["avg of imp"] = CI_featureimp.sum(axis=1, numeric_only=True)/63
    CI_featureimp = CI_featureimp[["feature","avg of imp"]]
    CI_featureimp = CI_featureimp.sort_values(by=['avg of imp'],ascending=False)
    CI_featureimp = CI_featureimp.reset_index(drop=True)
    CI_featureimp = CI_featureimp[:min(20,len(CI_featureimp))]
    print(CI_featureimp)    
    
    print("#=============all===============")
    shap.summary_plot(all_shap, all_data)
    print("#=============all===============")
    
    index_list = where(all_data,CI_featureimp["feature"].values)

    
    print("#=============part===============")
    plt.figure()
    shap.summary_plot(all_shap[:,index_list], all_data.iloc[:,index_list], show=False)
    if record:
        plt.savefig(plotname+'_shap_summary_plot.jpg', format='jpg', dpi=300, bbox_inches='tight')
    plt.show()
    print("#=============part===============")    
    
    if record:
        condition = record_table.iloc[:, 2:].isnull().all(axis=1)
        condition = record_table.iloc[:, 2:].isnull().all(axis=1)
        if condition.any():
            empty_row_index = record_table[condition].index[0]
            record_table.iloc[empty_row_index, 2:] = [len(data.columns), 
                                      str(np.quantile(acc,0.5))+" ( "+str(np.quantile(acc,0.05))+" ~ "+str(np.quantile(acc,0.95))+" )",
                                      str(np.quantile(sen,0.5))+" ( "+str(np.quantile(sen,0.05))+" ~ "+str(np.quantile(sen,0.95))+" )",
                                      str(np.quantile(spe,0.5))+" ( "+str(np.quantile(spe,0.05))+" ~ "+str(np.quantile(spe,0.95))+" )",
                                      str(np.quantile(auc,0.5))+" ( "+str(np.quantile(auc,0.05))+" ~ "+str(np.quantile(auc,0.95))+" )",
                                      str(np.quantile(auprc,0.5))+" ( "+str(np.quantile(auprc,0.05))+" ~ "+str(np.quantile(auprc,0.95))+" )"]
    
    n = np.where(auc == np.quantile(auc,0.05))
    
    m_fpr = l_fpr[n[0][0]]
    m_tpr = l_tpr[n[0][0]]
    plt.plot(m_fpr,m_tpr)
    #plt.legend(loc=4)
    plt.xlabel("false positive rate")
    plt.ylabel("true positive rate")
    
    if record:
        plt.savefig(plotname+'_auroc.jpg', format='jpg', dpi=300, bbox_inches='tight')
    
    plt.show()
        
        
    m_precision = l_fpr[n[0][0]]
    m_recall = l_tpr[n[0][0]]
    plt.plot(m_precision,m_recall)
    #plt.legend(loc=4)
    plt.xlabel("precision")
    plt.ylabel("recall")
    
    if record:
        plt.savefig(plotname+'_auprc.jpg', format='jpg', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("sen:",np.quantile(sen,0.5),"(",np.quantile(sen,0.05),"~",np.quantile(sen,0.95),")")
    print("spe:",np.quantile(spe,0.5),"(",np.quantile(spe,0.05),"~",np.quantile(spe,0.95),")")
    print("auprc:",np.quantile(auprc,0.5),"(",np.quantile(auprc,0.05),"~",np.quantile(auprc,0.95),")")
    print("acc:",np.quantile(acc,0.5),"(",np.quantile(acc,0.05),"~",np.quantile(acc,0.95),")")
    print("auc:",np.quantile(auc,0.5),"(",np.quantile(auc,0.05),"~",np.quantile(auc,0.95),")")
    

In [ ]:
def RFE(data,y,model,n_features_to_select,step):
    while True:
        if len(data.columns)>n_features_to_select:
            model.fit(data, y)
            table = featureIMP(data,model,modeltype=modeltype)
            num = max(n_features_to_select,len(data.columns)-step)
            imp_feature = table["feature"][0:num]
            data = data[imp_feature]
            print("RFE select ",len(data.columns)," features.")
        else:
            break
    return(data)

def backward(data,y,model,n_features_to_select):
    model.fit(data, y)
    table = featureIMP(data,model,modeltype=modeltype)
    imp_feature = table["feature"][0:n_features_to_select]
    data = data[imp_feature]
    print("Backward select ",len(data.columns)," features.")
    return(data)

In [ ]:
from sklearn.utils import shuffle
from catboost import CatBoostRegressor
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE


from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier

import lightgbm as lgb

import shap

In [ ]:
#basemodel =RandomForestClassifier(max_depth=4)
#modeltype = "forest"
#
#basemodel =XGBClassifier(n_estimators=100, learning_rate= 0.015)
#modeltype = "xgb"

#basemodel = CatBoostClassifier(loss_function="Logloss",
#                           eval_metric="AUC",
#                           task_type="GPU",
#                           learning_rate=0.01,
#                           iterations=30,
#                           random_seed=42,
#                           od_type="Iter",
#                           depth=9,
#                           early_stopping_rounds=10)
#modeltype = "catboost"

#basemodel = lgb.LGBMClassifier(learning_rate=0.04,max_depth=3,num_leaves=6,reg_alpha=0.1,reg_lambda=1,colsample_bytree=0.5)
#modeltype = "lgbm"

## catboost

In [ ]:
#catboost會消耗大量記憶體，因此fusion前期需用簡單的模型跑，後期再改回較大的模型
#basemodel2 = CatBoostClassifier(loss_function="Logloss",
#                           eval_metric="AUC",
#                           task_type="CPU",
#                           learning_rate=0.01,
#                           iterations=30,
#                           random_seed=42,
#                           od_type="Iter",
#                           depth=3,
#                           early_stopping_rounds=10,
#                            used_ram_limit="20gb",
#                            leaf_estimation_method='Newton',
#                           thread_count=5,
#                            pinned_memory_size="20gb")
#modeltype = "catboost"
#clinical_target = [0,1,2,3,6,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
#target_num = 6

In [ ]:
selecter_basemodel =RandomForestClassifier(max_depth=4)
CI_basemodel =RandomForestClassifier(max_depth=4)
modeltype = "forest"
clinical_target = [0,1,2,3,6,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
target_num = 6
selecter = RFE

In [ ]:
record = False
folder_path = './record/'
csv_folder_path = './record table/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
#建立空的pandas，自訂欄位並讓後去資料放進去    
record_table = pd.DataFrame(columns=['target','combination','feature number', 'accuracy', 'sensitivity', 'specificity', 'auroc' , 'auprc'], index=range(36))
record_table['target'] = ['mRS']*9+['NIHSS']*9+['LVO']*9+['T/E']*9
record_table['combination'] = ["C","R","S","F","F+C","F+R","F+S","F+C+R","F+C+R+S"]*4
record_table

## mRS

### only C

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_C"
target = clinical_target
data,y = get_data(clinical,target_ls=target,target_num=target_num)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID","mRS_discharge"]), pd.Series(selected_data.columns.values)])
clinical_after_selection = [clinical.columns.get_loc(label) for label in imp_feature]

### only R

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=radiomics)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_radiomics = radiomics[imp_feature]

### only S

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,semantic=semantic)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_semantic = semantic[imp_feature]

### only F

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_F"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=fuse)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=10000 , step=40000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_fuse = fuse[imp_feature]

### F+C

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_F+C"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+R

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_F+R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## F+S

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_F+S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse,semantic=selected_semantic)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_F+C+R"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R+S

In [ ]:
plotname = folder_path+modeltype+"_mRS_RFE_F+C+R+S"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,semantic=selected_semantic,fuse=selected_fuse)
y = (y>2)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## NIHSS

In [ ]:
clinical_target2 = [0,1,2,3,7,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
target_num = 7

### only C

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_C"
target = clinical_target2
data,y = get_data(clinical,target_ls=target,target_num=target_num)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID","NIHSS_discharge"]), pd.Series(selected_data.columns.values)])
clinical_after_selection = [clinical.columns.get_loc(label) for label in imp_feature]

### only R

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=radiomics)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_radiomics = radiomics[imp_feature]

### only S

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,semantic=semantic)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_semantic = semantic[imp_feature]

### only F

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_F"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=fuse)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=10000 , step=40000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_fuse = fuse[imp_feature]

### F+C

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_F+C"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+R

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_F+R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## F+S

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_F+S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse,semantic=selected_semantic)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_F+C+R"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R+S

In [ ]:
plotname = folder_path+modeltype+"_NIHSS_RFE_F+C+R+S"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,semantic=selected_semantic,fuse=selected_fuse)
y = (y>4)*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## LVO

In [ ]:
clinical_target3 = [0,1,2,3,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
target_num = 9

### only C

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_C"
target = clinical_target3
data,y = get_data(clinical,target_ls=target,target_num=target_num)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID","LVO-index"]), pd.Series(selected_data.columns.values)])
clinical_after_selection = [clinical.columns.get_loc(label) for label in imp_feature]

### only R

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=radiomics)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_radiomics = radiomics[imp_feature]

### only S

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,semantic=semantic)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_semantic = semantic[imp_feature]

### only F

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_F"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=fuse)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=10000 , step=40000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_fuse = fuse[imp_feature]

### F+C

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_F+C"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+R

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_F+R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## F+S

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_F+S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse,semantic=selected_semantic)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_F+C+R"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R+S

In [ ]:
plotname = folder_path+modeltype+"_LVO_RFE_F+C+R+S"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,semantic=selected_semantic,fuse=selected_fuse)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## T/E

In [ ]:
clinical_target4 = [0,1,2,3,8,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]
target_num = 8

### only C

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_C"
target = clinical_target4
data,y = get_data(clinical,target_ls=target,target_num=target_num)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID","T/E"]), pd.Series(selected_data.columns.values)])
clinical_after_selection = [clinical.columns.get_loc(label) for label in imp_feature]

### only R

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=radiomics)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_radiomics = radiomics[imp_feature]

### only S

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,semantic=semantic)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_semantic = semantic[imp_feature]

### only F

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_F"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=fuse)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=10000 , step=40000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=950 , step=2000)
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
imp_feature = pd.concat([pd.Series(["ID"]), pd.Series(selected_data.columns.values)])
selected_fuse = fuse[imp_feature]

### F+C

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_F+C"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+R

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_F+R"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

## F+S

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_F+S"
target = [0,target_num]
data,y = get_data(clinical,target_ls=target,target_num=target_num,fuse=selected_fuse,semantic=selected_semantic)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_F+C+R"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,fuse=selected_fuse)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

### F+C+R+S

In [ ]:
plotname = folder_path+modeltype+"_TE_RFE_F+C+R+S"
target = clinical_after_selection
data,y = get_data(clinical,target_ls=target,target_num=target_num,radiomics=selected_radiomics,semantic=selected_semantic,fuse=selected_fuse)
y = (y=="E")*1

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(data,y,selecter_model,n_features_to_select=50 , step=150)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=50 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=45 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=40 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=35 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=30 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=25 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=20 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=15 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
selecter_model = selecter_basemodel
selected_data = selecter(selected_data,y,selecter_model,n_features_to_select=10 , step=1)
CI_model = CI_basemodel
CI_analyse(selected_data,y,CI_model,modeltype=modeltype,record=record,plotname = plotname)

In [ ]:
if record:
    record_table.to_csv(csv_folder_path+modeltype+"_RFE.csv",index=False)
    record_table